## Data set preparation for ML model generation for WM apothecia prediction in irrigated environments


### Set up/check environment

In [1]:
# Check environment
!conda info
# active environment should be WM_ML


     active environment : WM_ML
    active env location : /opt/anaconda3/envs/WM_ML
            shell level : 1
       user config file : /Users/jilliancheck/.condarc
 populated config files : /Users/jilliancheck/.condarc
          conda version : 24.11.3
    conda-build version : 24.5.1
         python version : 3.12.4.final.0
                 solver : libmamba (default)
       virtual packages : __archspec=1=m1
                          __conda=24.11.3=0
                          __osx=15.0=0
                          __unix=0=0
       base environment : /opt/anaconda3  (writable)
      conda av data dir : /opt/anaconda3/etc/conda
  conda av metadata url : None
           channel URLs : https://repo.anaconda.com/pkgs/main/osx-arm64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/osx-arm64
                          https://repo.anaconda.com/pkgs/r/noarch
          package cache : /opt/anaconda3/pkgs
     

In [2]:
# Import packages 
import pandas as pd
import random
import numpy as np
import sklearn
from sklearn import datasets
from datetime import datetime
from itertools import cycle
import glob2
import os
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import scipy.stats as stats

# set the number of maximum displayed rows for printed dataframes to 1000
pd.set_option('display.max_rows', 1000)

### CSV load in and dataframe merging

In [3]:
# declare path with 2021 weather data
path = '/Users/jilliancheck/OneDrive - Michigan State University/Documents/Work/White mold/Field season 2021/Weather data/Moving averages/'

# reading all the excel files - appending a column with the filename
filenames = glob2.glob(path + "/*.csv")
files = filenames
weather21_df = pd.DataFrame()
for file in files:
    data = pd.read_csv(file)
    data['loccode'] = os.path.basename(file)
    weather21_df = pd.concat([weather21_df, data], axis=0)
#weather21_df.to_csv('merged_weather21_files.csv', index=False)
weather21_df['loccode'] = weather21_df['loccode'].str.removesuffix('_30.csv')
#display(weather21_df)


In [4]:
# load in 2021 apothecia density data
apo21_df = pd.read_csv('/Users/jilliancheck/OneDrive - Michigan State University/Documents/Work/White mold/Field season 2021/Apothecia data/apo21.csv')
#display(apo21_df)

In [5]:
# declare path with 2022 weather data
path = '/Users/jilliancheck/OneDrive - Michigan State University/Documents/Work/White mold/Field season 2022/Weather data/Moving averages/'

# reading all the excel files - appending a column with the filename
filenames = glob2.glob(path + "/*.csv")
files = filenames
weather22_df = pd.DataFrame()
for file in files:
    data = pd.read_csv(file)
    data['loccode'] = os.path.basename(file)
    weather22_df = pd.concat([weather22_df, data], axis=0)
#weather22_df.to_csv('merged_weather22_files.csv', index=False)
weather22_df['loccode'] = weather22_df['loccode'].str.removesuffix('_30.csv')
display(weather22_df)


,MA_MeanAT,MA_MeanDP,MA_MeanRH,MA_MeanWS,MA_MeanIR,MA_MeanRH86,MA_MeanATD,MA_MeanRF_IBM,MA_MeanRF_LOC,MA_MeanPC_IBM,...,MA_SumPC_IBM,MA_SumPC_LOC,MA_SumLW_count,MA_SumSM_IR,MA_SumST_IR,MA_SumWet_count,MA_SumDry_count,Month,Day,loccode
0,9.087500,7.062500,88.037500,4.388889,0.0,0.625000,0.333333,0.616667,0.616667,0.616667,...,14.8,14.800,NaN,NaN,NaN,1,0,5,1,MRCPOT22
1,8.729167,6.195833,84.914583,4.144676,0.0,0.520833,0.166667,0.308333,0.308333,0.308333,...,14.8,14.800,NaN,NaN,NaN,1,1,5,2,MRCPOT22
2,8.405556,6.366667,87.687500,4.025463,0.0,0.625000,0.111111,0.387500,0.387500,0.387500,...,27.9,27.900,NaN,NaN,NaN,2,1,5,3,MRCPOT22
3,8.813542,6.010417,84.046875,3.765046,0.0,0.562500,0.208333,0.294792,0.294792,0.294792,...,28.3,28.300,NaN,NaN,NaN,4,1,5,4,MRCPOT22
4,8.971667,5.611667,81.366667,3.395370,0.0,0.508333,0.283333,0.235833,0.235833,0.235833,...,28.3,28.300,NaN,NaN,NaN,4,2,5,5,MRCPOT22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,17.034444,13.291806,80.687500,2.108295,0.0,0.501389,0.784722,0.105972,0.071614,0.105972,...,76.3,51.562,1482.0,NaN,NaN,16,61,9,29,CHR22
152,16.699722,13.006250,80.874861,2.022492,0.0,0.504167,0.773611,0.105972,0.071614,0.105972,...,76.3,51.562,1421.0,NaN,NaN,16,62,9,30,CHR22
153,16.374028,12.742083,81.079167,2.071335,0.0,0.508333,0.770833,0.105972,0.071967,0.105972,...,76.3,51.816,1333.0,NaN,NaN,16,63,10,1,CHR22
154,16.035972,12.367639,80.883889,2.130826,0.0,0.504167,0.779167,0.105972,0.071967,0.105972,...,76.3,51.816,1245.0,NaN,NaN,16,64,10,2,CHR22


In [6]:
# load in 2022 apothecia density data
apo22_df = pd.read_csv('/Users/jilliancheck/OneDrive - Michigan State University/Documents/Work/White mold/Field season 2022/Apothecia raw data/apo22.csv')
#display(apo22_df)

In [7]:
# declare path with 2023 weather data
path = '/Users/jilliancheck/OneDrive - Michigan State University/Documents/Work/White mold/Field season 2023/Weather data/Moving averages - using irrigation log data/'

# reading all the excel files - appending a column with the filename
filenames = glob2.glob(path + "/*.csv")
files = filenames
weather23_df = pd.DataFrame()
for file in files:
    data = pd.read_csv(file)
    data['loccode'] = os.path.basename(file)
    weather23_df = pd.concat([weather23_df, data], axis=0)
#weather22_df.to_csv('merged_weather22_files.csv', index=False)
weather23_df['loccode'] = weather23_df['loccode'].str.removesuffix('_30.csv')
display(weather23_df)


,MA_MeanAT,MA_MeanDP,MA_MeanRH,MA_MeanWS,MA_MeanIR,MA_MeanRH86,MA_MeanATD,MA_MeanRF_IBM,MA_MeanRF_LOC,MA_MeanPC_IBM,...,MA_SumPC_IBM,MA_SumPC_LOC,MA_SumLW_count,MA_SumSM_IR,MA_SumST_IR,MA_SumWet_count,MA_SumDry_count,Month,Day,loccode
0,3.616667,1.466667,86.458333,5.055556,0.000000,0.708333,-6.661340e-16,0.266667,0.266667,0.266667,...,6.40,6.40,NaN,NaN,NaN,1,0,5,1,MRCDB23
1,3.410417,1.127083,85.608333,5.853588,0.000000,0.625000,-3.330670e-16,0.231250,0.231250,0.231250,...,11.10,11.10,NaN,NaN,NaN,3,0,5,2,MRCDB23
2,4.840278,1.734722,81.558333,5.492284,0.000000,0.513889,8.333333e-02,0.156944,0.156944,0.156944,...,11.30,11.30,NaN,NaN,NaN,6,0,5,3,MRCDB23
3,5.946875,1.981250,78.164583,4.383681,0.000000,0.468750,1.979167e-01,0.117708,0.117708,0.117708,...,11.30,11.30,NaN,NaN,NaN,6,1,5,4,MRCDB23
4,7.581667,2.677500,74.350833,4.223148,0.000000,0.383333,2.916667e-01,0.094167,0.094167,0.094167,...,11.30,11.30,NaN,NaN,NaN,6,3,5,5,MRCDB23
5,8.784722,3.131250,71.038889,4.243056,0.000000,0.319444,3.888889e-01,0.107639,0.107639,0.107639,...,15.50,15.50,NaN,NaN,NaN,7,3,5,6,MRCDB23
6,9.713095,4.356548,72.332738,4.113757,0.000000,0.339286,4.761905e-01,0.100000,0.100000,0.100000,...,16.80,16.80,NaN,NaN,NaN,9,3,5,7,MRCDB23
7,10.132812,4.845313,72.572917,4.010995,0.000000,0.343750,5.156250e-01,0.087500,0.087500,0.087500,...,16.80,16.80,NaN,NaN,NaN,9,4,5,8,MRCDB23
8,10.606944,4.555093,70.020833,3.686214,0.000000,0.310185,5.370370e-01,0.077778,0.077778,0.077778,...,16.80,16.80,NaN,NaN,NaN,9,6,5,9,MRCDB23
9,11.152500,4.587917,68.365417,3.514352,0.000000,0.291667,5.583333e-01,0.070000,0.070000,0.070000,...,16.80,16.80,NaN,NaN,NaN,9,9,5,10,MRCDB23


In [8]:
# load in 2023 apothecia density data
apo23_df = pd.read_csv('/Users/jilliancheck/OneDrive - Michigan State University/Documents/Work/White mold/Field season 2023/Apothecia raw data/apo23.csv')
display(apo23_df)

,ID,Date,Month,Day,Year,loccode,crop,soil type,spacing (in),spacing (m),obs count,apo count,apo density (apo/m2),canopy_avg
0,1,7/5/2023,7,5,2023,MRCSOY3023,soy,loamy sand,30,0.76,16,0,0.00,29.96
1,2,7/5/2023,7,5,2023,MRCSOY1523,soy,loamy sand,15,0.38,20,0,0.00,53.64
2,3,7/5/2023,7,5,2023,MRCDB23,dry bean,loamy sand,20,0.51,20,0,0.00,15.66
3,4,7/5/2023,7,5,2023,MRCPOT23,potato,loamy sand,34,0.86,20,0,0.00,61.51
4,5,7/12/2023,7,12,2023,MRCSOY3023,soy,loamy sand,30,0.76,20,0,0.00,48.85
5,6,7/12/2023,7,12,2023,MRCSOY1523,soy,loamy sand,15,0.38,20,1,0.13,81.47
6,7,7/15/2023,7,15,2023,MRCDB23,dry bean,loamy sand,20,0.51,20,0,0.00,35.69
7,8,7/15/2023,7,15,2023,MRCPOT23,potato,loamy sand,34,0.86,20,2,0.12,94.11
8,9,7/15/2023,7,15,2023,MRCSOY1523,soy,loamy sand,15,0.38,20,0,0.00,93.88
9,10,7/15/2023,7,15,2023,MRCSOY3023,soy,loamy sand,30,0.76,20,5,0.33,60.55


In [9]:
# merge weather and apothecia data into single data frame for both years
final21_df = apo21_df.merge(weather21_df, how = 'left', left_on = ['loccode', 'Month', 'Day'], right_on = ['loccode', 'Month', 'Day'])
#display(final21_df)
final22_df = apo22_df.merge(weather22_df, how = 'left', left_on = ['loccode', 'Month', 'Day'], right_on = ['loccode', 'Month', 'Day'])
#display(final22_df)
final23_df = apo23_df.merge(weather23_df, how = 'left', left_on = ['loccode', 'Month', 'Day'], right_on = ['loccode', 'Month', 'Day'])
display(final23_df)

,ID,Date,Month,Day,Year,loccode,crop,soil type,spacing (in),spacing (m),...,MA_SumATD,MA_SumRF_IBM,MA_SumRF_LOC,MA_SumPC_IBM,MA_SumPC_LOC,MA_SumLW_count,MA_SumSM_IR,MA_SumST_IR,MA_SumWet_count,MA_SumDry_count
0,1,7/5/2023,7,5,2023,MRCSOY3023,soy,loamy sand,30,0.76,...,543,108.7,137.79,178.55,207.64,NaN,5078.410900,6815.726833,20,25
1,2,7/5/2023,7,5,2023,MRCSOY1523,soy,loamy sand,15,0.38,...,543,108.7,137.79,178.55,207.64,NaN,5078.410900,6815.726833,20,25
2,3,7/5/2023,7,5,2023,MRCDB23,dry bean,loamy sand,20,0.51,...,543,108.7,137.79,162.04,191.13,938.0,4683.370183,6807.049317,18,42
3,4,7/5/2023,7,5,2023,MRCPOT23,potato,loamy sand,34,0.86,...,543,108.7,137.79,178.55,207.64,NaN,2921.132850,4132.824250,20,25
4,5,7/12/2023,7,12,2023,MRCSOY3023,soy,loamy sand,30,0.76,...,550,157.9,127.63,215.05,184.78,NaN,7182.418650,9689.966933,22,24
5,6,7/12/2023,7,12,2023,MRCSOY1523,soy,loamy sand,15,0.38,...,550,157.9,127.63,215.05,184.78,NaN,7182.418650,9689.966933,22,24
6,7,7/15/2023,7,15,2023,MRCDB23,dry bean,loamy sand,20,0.51,...,554,188.5,196.85,234.22,242.57,1565.0,7711.386233,11509.019317,27,22
7,8,7/15/2023,7,15,2023,MRCPOT23,potato,loamy sand,34,0.86,...,554,188.5,196.85,226.60,234.95,NaN,5638.695535,8888.403300,20,25
8,9,7/15/2023,7,15,2023,MRCSOY1523,soy,loamy sand,15,0.38,...,554,188.5,196.85,226.60,234.95,NaN,9372.767000,11145.754633,20,25
9,10,7/15/2023,7,15,2023,MRCSOY3023,soy,loamy sand,30,0.76,...,554,188.5,196.85,226.60,234.95,NaN,9372.767000,11145.754633,20,25


In [11]:
# merge all years of weather and apothecia data into single data frame
dfs = [final21_df, final22_df, final23_df]
final_df = pd.concat(dfs, ignore_index=True)

# create new column using logical test on apothecia density
final_df['apo_thrs'] = [1 if val > 0.176 else 0 for val in final_df['apo density (apo/m2)']]

# move apothecia threshold column to the right of the dataset for exlusion during PCA
    # Specify the column to move and its desired position
column_to_move = 'apo_thrs'
position_after_column = 'spacing (in)'
    # Remove the column from its current position
column = final_df.pop(column_to_move)
    # Get the index of the column to move after
index = final_df.columns.get_loc(position_after_column)
    # Insert the column at the desired position
final_df.insert(index + 1, column.name, column)

# write out for figure building
final_df.to_csv('/Users/jilliancheck/Library/CloudStorage/OneDrive-MichiganStateUniversity/Documents/Work/White mold/Modeling/jupyter notebooks/!Finalizing - 2024/Data/FigureBuilding.csv', index=False)

# designate target variable (apothecia density)
final_df = final_df.rename(columns = {"apo_thrs" : "target"})
display(final_df)

,ID,Date,Month,Day,Year,loccode,crop,soil type,spacing (in),target,...,MA_SumATD,MA_SumRF_IBM,MA_SumRF_LOC,MA_SumPC_IBM,MA_SumPC_LOC,MA_SumLW_count,MA_SumSM_IR,MA_SumST_IR,MA_SumWet_count,MA_SumDry_count
0,1,7/1/2021,7,1,2021,CHR21,soy,loam,15,0,...,532,249.2,249.200,NaN,NaN,NaN,NaN,NaN,0,0
1,19,7/21/2021,7,21,2021,CHR21,soy,loam,15,0,...,562,231.8,231.800,NaN,NaN,NaN,NaN,NaN,0,0
2,35,8/6/2021,8,6,2021,CHR21,soy,loam,15,1,...,568,93.8,70.094,58.504,34.798,180.0,3117.916742,4970.720308,9,15
3,40,8/10/2021,8,10,2021,CHR21,soy,loam,15,1,...,525,87.9,59.648,68.638,40.386,229.0,4218.271158,6762.569275,15,16
4,47,8/17/2021,8,17,2021,CHR21,soy,loam,15,0,...,517,122.8,42.356,134.038,53.594,366.0,5779.277750,9208.189375,24,31
5,5,7/9/2021,7,9,2021,CRY21,dry bean,loamy sand,20,0,...,566,214.3,214.300,NaN,NaN,NaN,NaN,NaN,0,0
6,12,7/17/2021,7,17,2021,CRY21,dry bean,loamy sand,20,0,...,595,233.5,233.500,NaN,NaN,NaN,NaN,NaN,0,0
7,20,7/26/2021,7,26,2021,CRY21,dry bean,loamy sand,20,1,...,563,86.6,74.840,17.856,6.096,77.0,791.432858,1333.557258,1,4
8,33,8/5/2021,8,5,2021,CRY21,dry bean,loamy sand,20,1,...,603,77.6,59.940,52.204,34.544,174.0,2934.870167,4677.423125,6,16
9,41,8/12/2021,8,12,2021,CRY21,dry bean,loamy sand,20,1,...,542,68.5,28.256,93.838,53.594,307.0,4805.097617,7620.721658,22,17


### Data description/summary statistics

In [20]:
# show summary statistics for each variable
pd.set_option('display.max_columns', None)
final_df.describe()

# remove 'nonsense' variables (zero variance variables)
    # MA_MinIR, MA_MinRF_IBM, MA_MinRF_LOC, MA_MinPC_IBM, MA_MinPC_LOC, MA_MinWet_count, MA_MinDry_count
final_df = final_df.drop(['MA_MaxATD', 'MA_MinIR', 'MA_MinRF_IBM', 'MA_MinRF_LOC', 'MA_MinPC_IBM', 'MA_MinPC_LOC', 'MA_MinWet_count', 'MA_MinDry_count'], axis = 1)
# remove non-predictor information (descriptors for observations)
    # Month, Day, Year, loccode, spacing (in), obs count, apo count
final_df = final_df.drop(['ID', 'Month', 'Day', 'Year', 'loccode', 'spacing (in)', 'obs count', 'apo count', 'apo density (apo/m2)'], axis = 1)

final_df.describe()

,target,spacing (m),canopy_avg,MA_MeanAT,MA_MeanDP,MA_MeanRH,MA_MeanWS,MA_MeanIR,MA_MeanRH86,MA_MeanATD,MA_MeanRF_IBM,MA_MeanRF_LOC,MA_MeanPC_IBM,MA_MeanPC_LOC,MA_MeanLW_count,MA_MeanSM_IR,MA_MeanST_IR,MA_MeanWet_count,MA_MeanDry_count,MA_MaxAT,MA_MaxDP,MA_MaxRH,MA_MaxWS,MA_MaxIR,MA_MaxRH86,MA_MaxRF_IBM,MA_MaxRF_LOC,MA_MaxPC_IBM,MA_MaxPC_LOC,MA_MaxLW_count,MA_MaxSM_IR,MA_MaxST_IR,MA_MaxWet_count,MA_MaxDry_count,MA_MinAT,MA_MinDP,MA_MinRH,MA_MinWS,MA_MinRH86,MA_MinATD,MA_MinSM_IR,MA_MinST_IR,MA_SumAT,MA_SumDP,MA_SumRH,MA_SumWS,MA_SumIR,MA_SumRH86,MA_SumATD,MA_SumRF_IBM,MA_SumRF_LOC,MA_SumPC_IBM,MA_SumPC_LOC,MA_SumLW_count,MA_SumSM_IR,MA_SumST_IR,MA_SumWet_count,MA_SumDry_count
count,152.000000,152.000000,150.000000,152.000000,152.000000,152.000000,152.000000,148.000000,152.000000,152.000000,152.000000,152.000000,148.000000,148.000000,68.000000,148.000000,148.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,148.000000,152.000000,152.000000,152.000000,148.000000,148.000000,68.000000,148.000000,148.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,152.000000,148.000000,148.000000,152.000000,152.000000,152.000000,152.000000,148.000000,152.000000,152.000000,152.000000,152.000000,148.000000,148.000000,68.000000,148.000000,148.000000,152.000000,152.000000
mean,0.552632,0.598033,83.771267,21.086640,15.302686,72.823987,2.666421,0.305014,0.348070,0.769608,0.176721,0.159231,0.474757,0.451709,1.229479,15.343343,20.914490,1.022368,0.795614,26.500680,17.667807,95.421009,4.845736,4.543404,0.921930,2.240877,2.856241,6.577909,7.212952,5.530887,16.876542,23.620159,4.546053,3.631579,15.435395,12.803180,49.369627,0.735630,0.010526,0.257675,14.235350,18.588271,15179.565132,11015.801316,52421.578289,1919.376827,165.199541,250.559211,554.013158,127.150000,114.565211,274.120486,261.195568,737.970588,8585.060448,11364.405217,30.671053,23.868421
std,0.498866,0.191939,20.848255,0.676574,1.266657,4.585063,0.321195,0.791450,0.073650,0.048039,0.087564,0.101326,0.816164,0.816068,0.679055,3.573484,1.676801,0.817971,0.432248,0.844504,1.080624,2.638034,0.454068,10.361871,0.088409,1.168162,2.179986,10.484051,10.470227,2.019508,3.851136,2.040718,2.626505,2.121115,0.824533,1.478619,5.249073,0.154274,0.024690,0.106518,3.388989,1.863191,488.935173,913.304086,3308.899011,230.838840,400.167598,53.062219,34.601992,62.986319,72.892947,400.862156,400.245577,521.916445,3603.727125,3839.783370,24.539137,12.967450
min,0.000000,0.380000,14.260000,19.224861,12.082361,60.204444,1.983796,0.000000,0.163889,0.655556,0.048889,0.015875,0.016667,0.014111,0.444444,8.834536,16.233096,0.000000,0.000000,24.373333,14.366667,88.283333,3.942593,0.000000,0.666667,0.620000,0.245533,0.233333,0.254000,2.928571,9.165625,19.199393,0.000000,0.000000,12.866667,8.933333,36.890000,0.362963,0.000000,0.066667,8.518625,13.399791,13841.900000,8699.300000,43347.200000,1428.333333,0.000000,118.000000,472.000000,35.200000,11.430000,1.200000,1.016000,30.000000,508.732500,752.022500,0.000000,0.000000
25%,0.000000,0.381000,78.955000,20.631072,14.703333,70.526458,2.382330,0.031309,0.316591,0.739266,0.087917,0.073203,0.177950,0.133879,0.713141,12.475257,20.187732,0.633333,0.466667,25.934167,17.213333,94.201667,4.471296,0.677333,0.900000,1.195833,1.140817,2.577395,2.219678,3.900000,14.304139,22.409195,3.000000,2.000000,15.083333,11.976667,45.320000,0.614352,0.000000,0.166667,11.548762,17.762770,14844.700000,10586.400000,50779.050000,1715.277778,19.050000,227.750000,531.750000,63.300000,52.206500,114.075000,89.723000,360.000000,6428.125217,9192.613242,19.000000,14.000000
50%,1.000000,0.510000,94.285000,21.065625,15.530939,74.822500,2.677276,0.059619,0.373792,0.775000,0.164861,0.139903,0.249493,0.222046,0.977362,15.095246,21.189023,0.783333,0.833333,26.520000,17.825000,96.226667,4.842130,1.278467,0.966667,1.920000,2.209800,3.546833,3.889500,5.014815,17.192964,23.618278,4.000000,4.000000,15.326667,13.225000,49.676667,0.712037,0.000000,0.266667,13.896750,18.5

In [21]:
# count the number of missing values for each predictor 

def count_missing_values(final_df):
    missing_values = final_df.isnull().sum()
    return missing_values

missing_values_count = count_missing_values(final_df)

print("Missing values count per column:")
print(missing_values_count)

Missing values count per column:
Date                 0
crop                 0
soil type            0
target               0
spacing (m)          0
canopy_avg           2
MA_MeanAT            0
MA_MeanDP            0
MA_MeanRH            0
MA_MeanWS            0
MA_MeanIR            4
MA_MeanRH86          0
MA_MeanATD           0
MA_MeanRF_IBM        0
MA_MeanRF_LOC        0
MA_MeanPC_IBM        4
MA_MeanPC_LOC        4
MA_MeanLW_count     84
MA_MeanSM_IR         4
MA_MeanST_IR         4
MA_MeanWet_count     0
MA_MeanDry_count     0
MA_MaxAT             0
MA_MaxDP             0
MA_MaxRH             0
MA_MaxWS             0
MA_MaxIR             4
MA_MaxRH86           0
MA_MaxRF_IBM         0
MA_MaxRF_LOC         0
MA_MaxPC_IBM         4
MA_MaxPC_LOC         4
MA_MaxLW_count      84
MA_MaxSM_IR          4
MA_MaxST_IR          4
MA_MaxWet_count      0
MA_MaxDry_count      0
MA_MinAT             0
MA_MinDP             0
MA_MinRH             0
MA_MinWS             0
MA_MinRH86           0
M

In [22]:
# the only data type missing in large quantities (>4 indices) is leaf wetness data
# for now, remove from data set

# filter the dataset to exclude columns containing the specified string
filtered_df = final_df.loc[:, ~final_df.columns.str.contains("LW")]

# display the filtered dataFrame to make sure LW columns are gone
display(filtered_df)

,Date,crop,soil type,target,spacing (m),canopy_avg,MA_MeanAT,MA_MeanDP,MA_MeanRH,MA_MeanWS,MA_MeanIR,MA_MeanRH86,MA_MeanATD,MA_MeanRF_IBM,MA_MeanRF_LOC,MA_MeanPC_IBM,MA_MeanPC_LOC,MA_MeanSM_IR,MA_MeanST_IR,MA_MeanWet_count,MA_MeanDry_count,MA_MaxAT,MA_MaxDP,MA_MaxRH,MA_MaxWS,MA_MaxIR,MA_MaxRH86,MA_MaxRF_IBM,MA_MaxRF_LOC,MA_MaxPC_IBM,MA_MaxPC_LOC,MA_MaxSM_IR,MA_MaxST_IR,MA_MaxWet_count,MA_MaxDry_count,MA_MinAT,MA_MinDP,MA_MinRH,MA_MinWS,MA_MinRH86,MA_MinATD,MA_MinSM_IR,MA_MinST_IR,MA_SumAT,MA_SumDP,MA_SumRH,MA_SumWS,MA_SumIR,MA_SumRH86,MA_SumATD,MA_SumRF_IBM,MA_SumRF_LOC,MA_SumPC_IBM,MA_SumPC_LOC,MA_SumSM_IR,MA_SumST_IR,MA_SumWet_count,MA_SumDry_count
0,7/1/2021,soy,loam,0,0.380,NaN,21.703442,15.086751,69.869269,2.558441,NaN,0.317029,0.739855,0.347168,0.347168,NaN,NaN,NaN,NaN,0.000000,0.000000,26.963333,17.510000,91.706667,4.853704,NaN,0.866667,4.033333,4.033333,NaN,NaN,NaN,NaN,0,0,16.226667,12.216667,47.543333,0.594444,0.000000,0.166667,NaN,NaN,15604.0,10845.8,50234.1,1838.555556,NaN,228,532,249.2,249.200,NaN,NaN,NaN,NaN,0,0
1,7/21/2021,soy,loam,0,0.380,99.28,21.551250,16.934306,76.845833,2.707832,NaN,0.397222,0.780556,0.321944,0.321944,NaN,NaN,NaN,NaN,0.000000,0.000000,26.133333,18.713333,93.943333,4.708333,NaN,0.966667,3.540000,3.540000,NaN,NaN,NaN,NaN,0,0,16.946667,14.813333,58.030000,0.618519,0.033333,0.300000,NaN,NaN,15516.9,12192.7,55329.0,1949.638889,NaN,286,562,231.8,231.800,NaN,NaN,NaN,NaN,0,0
2,8/6/2021,soy,loam,1,0.380,96.89,21.311806,16.241250,75.237500,2.089853,0.140220,0.381944,0.788889,0.130278,0.097353,0.229361,0.149493,13.555754,21.311371,0.300000,0.500000,26.476667,18.053333,94.006667,3.942593,1.828800,0.933333,1.746667,1.604667,2.668800,1.862667,15.344592,22.310800,2,4,16.096667,14.186667,54.020000,0.362963,0.033333,0.333333,12.561304,20.255709,15344.5,11693.7,54171.0,1504.694444,32.004,275,568,93.8,70.094,58.504,34.798,3117.916742,4970.720308,9,15
3,8/10/2021,soy,loam,1,0.380,NaN,22.037639,16.919444,75.121944,2.227199,0.126296,0.388889,0.729167,0.122083,0.082844,0.209628,0.134360,13.604341,21.542087,0.500000,0.533333,27.400000,18.803333,94.046667,4.107407,1.724526,0.966667,1.636667,1.403133,2.545579,1.751263,15.213627,22.566536,3,4,16.803333,14.903333,53.250000,0.487963,0.033333,0.200000,12.593995,20.549204,15867.1,12182.0,54087.8,1603.583333,37.338,280,525,87.9,59.648,68.638,40.386,4218.271158,6762.569275,15,16
4,8/17/2021,soy,loam,0,0.380,97.57,22.052639,16.547500,73.497083,2.072184,0.092294,0.344444,0.718056,0.170556,0.058828,0.290369,0.126408,13.678102,21.625430,0.800000,1.033333,27.763333,18.646667,93.970000,3.984259,1.260231,0.966667,1.926667,1.183533,2.967923,1.553308,15.318765,22.664764,5,5,16.380000,14.286667,50.033333,0.437963,0.000000,0.133333,12.554673,20.623318,15877.9,11914.2,52917.9,1491.972222,37.338,248,517,122.8,42.356,134.038,53.594,5779.277750,9208.189375,24,31
5,7/9/2021,dry bean,loamy sand,0,0.510,39.33,20.788587,15.161322,73.061401,2.884318,NaN,0.347524,0.787138,0.298635,0.298635,NaN,NaN,NaN,NaN,0.000000,0.000000,25.590000,17.586667,92.643333,4.961111,NaN,0.866667,3.263333,3.263333,NaN,NaN,NaN,NaN,0,0,15.670000,12.213333,53.260000,0.940741,0.100000,0.366667,NaN,NaN,14944.9,10899.6,52534.7,2072.388889,NaN,250,566,214.3,214.300,NaN,NaN,NaN,NaN,0,0
6,7/17/2021,dry bean,loamy sand,0,0.510,74.76,20.680531,16.247295,77.628484,3.009086,NaN,0.408635,0.827415,0.325302,0.325302,NaN,NaN,NaN,NaN,0.000000,0.000000,24.763333,18.396667,94.270000,5.164815,NaN,0.966667,3.840000,3.840000,NaN,NaN,NaN,NaN,0,0,16.210000,13.556667,59.830000,0.974074,0.100000,0.500000,NaN,NaN,14867.1,11681.5,55823.0,2162.222222,NaN,294,595,233.5,233.500,NaN,NaN,NaN,NaN,0,0
7,7/26/2021,dry bean,loamy sand,1,0.510,92.55,21.771667,16.786944,75.555556,2.747222,0.042333,0.355556,0.781944,0.120278,0.103944,0.212571,0.072571,13.357698,22.298266,0.033333,0.133333,26.343333,18.836667,93.476667,4.750926,0.889000,0.933333,1.813333,1.689533,2.389000,0.952500,14.355727,23.243283,1,2,17.086667,14.463333,55.970000,0.869444,0.033333,0.4333

In [15]:
# reassess where/how many data are incomplete

# count the number of missing values for each predictor 

def count_missing_values(filtered_df):
    missing_values = filtered_df.isnull().sum()
    return missing_values

missing_values_count = count_missing_values(filtered_df)

print("Missing values count per column:")
print(missing_values_count)

Missing values count per column:
Date                0
crop                0
soil type           0
target              0
spacing (m)         0
canopy_avg          2
MA_MeanAT           0
MA_MeanDP           0
MA_MeanRH           0
MA_MeanWS           0
MA_MeanIR           4
MA_MeanRH86         0
MA_MeanATD          0
MA_MeanRF_IBM       0
MA_MeanRF_LOC       0
MA_MeanPC_IBM       4
MA_MeanPC_LOC       4
MA_MeanSM_IR        4
MA_MeanST_IR        4
MA_MeanWet_count    0
MA_MeanDry_count    0
MA_MaxAT            0
MA_MaxDP            0
MA_MaxRH            0
MA_MaxWS            0
MA_MaxIR            4
MA_MaxRH86          0
MA_MaxRF_IBM        0
MA_MaxRF_LOC        0
MA_MaxPC_IBM        4
MA_MaxPC_LOC        4
MA_MaxSM_IR         4
MA_MaxST_IR         4
MA_MaxWet_count     0
MA_MaxDry_count     0
MA_MinAT            0
MA_MinDP            0
MA_MinRH            0
MA_MinWS            0
MA_MinRH86          0
MA_MinATD           0
MA_MinSM_IR         4
MA_MinST_IR         4
MA_SumAT            0

In [16]:
# only 6 data lines are incomplete - remove for PCA, and reset indexes

filtered_df = filtered_df.dropna().reset_index()
filtered_df.describe()

# for modeling, 
    # either impute missing data OR 
    # remove the limited number of observations with missing data
        # find out which algorithms can handle missing data

,index,target,spacing (m),canopy_avg,MA_MeanAT,MA_MeanDP,MA_MeanRH,MA_MeanWS,MA_MeanIR,MA_MeanRH86,MA_MeanATD,MA_MeanRF_IBM,MA_MeanRF_LOC,MA_MeanPC_IBM,MA_MeanPC_LOC,MA_MeanSM_IR,MA_MeanST_IR,MA_MeanWet_count,MA_MeanDry_count,MA_MaxAT,MA_MaxDP,MA_MaxRH,MA_MaxWS,MA_MaxIR,MA_MaxRH86,MA_MaxRF_IBM,MA_MaxRF_LOC,MA_MaxPC_IBM,MA_MaxPC_LOC,MA_MaxSM_IR,MA_MaxST_IR,MA_MaxWet_count,MA_MaxDry_count,MA_MinAT,MA_MinDP,MA_MinRH,MA_MinWS,MA_MinRH86,MA_MinATD,MA_MinSM_IR,MA_MinST_IR,MA_SumAT,MA_SumDP,MA_SumRH,MA_SumWS,MA_SumIR,MA_SumRH86,MA_SumATD,MA_SumRF_IBM,MA_SumRF_LOC,MA_SumPC_IBM,MA_SumPC_LOC,MA_SumSM_IR,MA_SumST_IR,MA_SumWet_count,MA_SumDry_count
count,147.000000,147.000000,147.00000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.00000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000,147.000000
mean,77.965986,0.564626,0.60368,84.029388,21.077604,15.276593,72.766797,2.666049,0.306230,0.347261,0.769499,0.173105,0.155287,0.476560,0.453868,15.355173,20.910221,1.053741,0.819048,26.511927,17.649637,95.492404,4.848684,4.562580,0.921769,2.206122,2.844005,6.605339,7.250106,16.887854,23.627326,4.680272,3.727891,15.403560,12.778095,49.199116,0.736054,0.00907,0.256009,14.246516,18.574931,15173.427891,10997.280272,52381.437415,1919.176115,166.069347,249.979592,553.945578,124.558503,111.737850,275.518327,262.697673,8614.766497,11395.710223,31.612245,24.571429
std,42.638720,0.497501,0.19232,20.682907,0.679213,1.271210,4.623227,0.322734,0.794017,0.074496,0.048412,0.085440,0.099084,0.818658,0.818434,3.582792,1.681727,0.812708,0.418175,0.838570,1.089603,2.647285,0.456703,10.394661,0.089447,1.161627,2.209077,10.514563,10.496228,3.861834,2.045825,2.556181,2.069206,0.815403,1.481229,5.193677,0.152602,0.02260,0.105625,3.397842,1.862456,490.825522,916.633523,3336.634481,231.976413,401.395288,53.672455,34.875273,61.476005,71.297496,401.870528,401.195134,3597.819224,3833.912917,24.381233,12.545261
min,2.000000,0.000000,0.38000,14.260000,19.224861,12.082361,60.204444,1.983796,0.000000,0.163889,0.655556,0.048889,0.015875,0.016667,0.014111,8.834536,16.233096,0.033333,0.000000,24.373333,14.366667,88.283333,3.942593,0.000000,0.666667,0.620000,0.245533,0.233333,0.254000,9.165625,19.199393,1.000000,0.000000,12.866667,8.933333,36.890000,0.362963,0.00000,0.066667,8.518625,13.399791,13841.900000,8699.300000,43347.200000,1428.333333,0.000000,118.000000,472.000000,35.200000,11.430000,1.200000,1.016000,508.732500,752.022500,1.000000,0.000000
25%,41.500000,0.000000,0.38100,79.695000,20.608889,14.608092,70.512500,2.382330,0.030868,0.313194,0.738889,0.087917,0.070549,0.177892,0.133703,12.426911,20.184054,0.650000,0.500000,25.965000,17.213333,94.233333,4.471296,0.668867,0.900000,1.185000,1.031767,2.597733,2.253074,14.284764,22.405805,3.000000,3.000000,15.083333,11.976667,45.320000,0.616667,0.00000,0.166667,11.522652,17.756782,14838.400000,10512.550000,50769.000000,1715.277778,19.050000,225.500000,532.000000,63.300000,50.339000,116.308000,92.837000,6452.948182,9299.549825,19.500000,15.000000
50%,78.000000,1.000000,0.51000,94.360000,21.064722,15.502295,74.822500,2.677276,0.058914,0.373611,0.775000,0.164861,0.139806,0.249944,0.222250,15.124355,21.178331,0.800000,0.833333,26.520000,17.816667,96.320000,4.835185,1.278467,0.966667,1.920000,2.209800,3.563333,3.894667,17.256500,23.632388,4.000000,4.000000,15.326667,13.220000,49.603333,0.712037,0.00000,0.266667,13.927500,18.583803,15166.600000,11145.600000,53872.200000,1927.638889,38.100000,269.000000,558.000000,118.700000,100.330000,175.000000,153.416000,8585.022510,12193.5244

In [17]:
# save final dataset as csv
filtered_df.to_csv('filtered_df.csv', index=False)

### Fall 2024: retesting for excluding 2023 data and LOCOMOS precip/irrig

In [18]:
path = '/Users/jilliancheck/OneDrive - Michigan State University/Documents/Work/White mold/modeling/jupyter notebooks/WM train test data Fall 24/'

In [19]:
# 21 and 22 RTMA
RTMA_2122_df = filtered_df.loc[:, ~filtered_df.columns.str.contains('LOC|PC|IR|count')]
RTMA_2122_df = RTMA_2122_df[ ~RTMA_2122_df['Date'].str.contains('2023')]

# check years
unique_values = RTMA_2122_df['Date'].unique()
print(unique_values)

# list headers
print(list(RTMA_2122_df))

# save final dataset as csv
RTMA_2122_df.to_csv(path + 'RTMA_22_df.csv', index=False)

['8/6/2021' '8/17/2021' '7/26/2021' '8/5/2021' '8/12/2021' '8/19/2021'
 '7/6/2021' '7/13/2021' '7/20/2021' '8/2/2021' '8/10/2021' '7/9/2021'
 '7/17/2021' '7/30/2021' '7/8/2021' '7/15/2021' '7/19/2021' '7/27/2021'
 '8/9/2021' '8/16/2021' '8/23/2021' '8/3/2021' '8/30/2021' '6/28/2022'
 '7/6/2022' '7/13/2022' '7/20/2022' '7/27/2022' '8/4/2022' '8/10/2022'
 '8/17/2022' '8/24/2022' '7/8/2022' '7/14/2022' '7/21/2022' '8/5/2022'
 '8/11/2022' '8/25/2022' '6/27/2022' '7/7/2022' '7/11/2022' '7/19/2022'
 '7/26/2022' '8/1/2022' '8/8/2022' '8/15/2022' '7/5/2022' '8/9/2022'
 '8/16/2022' '7/12/2022' '8/2/2022' '7/25/2022' '8/23/2022' '7/18/2022'
 '8/22/2022']
['index', 'Date', 'crop', 'soil type', 'target', 'spacing (m)', 'canopy_avg', 'MA_MeanAT', 'MA_MeanDP', 'MA_MeanRH', 'MA_MeanWS', 'MA_MeanRH86', 'MA_MeanATD', 'MA_MeanRF_IBM', 'MA_MaxAT', 'MA_MaxDP', 'MA_MaxRH', 'MA_MaxWS', 'MA_MaxRH86', 'MA_MaxRF_IBM', 'MA_MinAT', 'MA_MinDP', 'MA_MinRH', 'MA_MinWS', 'MA_MinRH86', 'MA_MinATD', 'MA_SumAT', 'MA_Su

In [20]:
# 21 and 22 RTMA + LOCOMOS - irri/pcp
RTMA_LX_2122_df = filtered_df.loc[:, ~filtered_df.columns.str.contains('LOC|PC|count')]
RTMA_LX_2122_df = RTMA_LX_2122_df.drop(['MA_MeanIR', 'MA_MaxIR', 'MA_SumIR'], axis = 1)
RTMA_LX_2122_df = RTMA_LX_2122_df[ ~RTMA_LX_2122_df['Date'].str.contains('2023')]

# check years
unique_values = RTMA_LX_2122_df['Date'].unique()
print(unique_values)

# list headers
print(list(RTMA_LX_2122_df))

# save final dataset as csv
RTMA_LX_2122_df.to_csv(path + 'RTMA_LX_22_df.csv', index=False)

['8/6/2021' '8/17/2021' '7/26/2021' '8/5/2021' '8/12/2021' '8/19/2021'
 '7/6/2021' '7/13/2021' '7/20/2021' '8/2/2021' '8/10/2021' '7/9/2021'
 '7/17/2021' '7/30/2021' '7/8/2021' '7/15/2021' '7/19/2021' '7/27/2021'
 '8/9/2021' '8/16/2021' '8/23/2021' '8/3/2021' '8/30/2021' '6/28/2022'
 '7/6/2022' '7/13/2022' '7/20/2022' '7/27/2022' '8/4/2022' '8/10/2022'
 '8/17/2022' '8/24/2022' '7/8/2022' '7/14/2022' '7/21/2022' '8/5/2022'
 '8/11/2022' '8/25/2022' '6/27/2022' '7/7/2022' '7/11/2022' '7/19/2022'
 '7/26/2022' '8/1/2022' '8/8/2022' '8/15/2022' '7/5/2022' '8/9/2022'
 '8/16/2022' '7/12/2022' '8/2/2022' '7/25/2022' '8/23/2022' '7/18/2022'
 '8/22/2022']
['index', 'Date', 'crop', 'soil type', 'target', 'spacing (m)', 'canopy_avg', 'MA_MeanAT', 'MA_MeanDP', 'MA_MeanRH', 'MA_MeanWS', 'MA_MeanRH86', 'MA_MeanATD', 'MA_MeanRF_IBM', 'MA_MeanSM_IR', 'MA_MeanST_IR', 'MA_MaxAT', 'MA_MaxDP', 'MA_MaxRH', 'MA_MaxWS', 'MA_MaxRH86', 'MA_MaxRF_IBM', 'MA_MaxSM_IR', 'MA_MaxST_IR', 'MA_MinAT', 'MA_MinDP', 'MA_Min

In [21]:
# 21 and 22 RTMA + LOCOMOS
RTMA_LOC_2122_df = filtered_df[ ~filtered_df['Date'].str.contains('2023')]

# check years
unique_values = RTMA_LOC_2122_df['Date'].unique()
print(unique_values)

# list headers
print(list(RTMA_LOC_2122_df))

# save final dataset as csv
RTMA_LOC_2122_df.to_csv(path + 'RTMA_LOC_22_df.csv', index=False)

['8/6/2021' '8/17/2021' '7/26/2021' '8/5/2021' '8/12/2021' '8/19/2021'
 '7/6/2021' '7/13/2021' '7/20/2021' '8/2/2021' '8/10/2021' '7/9/2021'
 '7/17/2021' '7/30/2021' '7/8/2021' '7/15/2021' '7/19/2021' '7/27/2021'
 '8/9/2021' '8/16/2021' '8/23/2021' '8/3/2021' '8/30/2021' '6/28/2022'
 '7/6/2022' '7/13/2022' '7/20/2022' '7/27/2022' '8/4/2022' '8/10/2022'
 '8/17/2022' '8/24/2022' '7/8/2022' '7/14/2022' '7/21/2022' '8/5/2022'
 '8/11/2022' '8/25/2022' '6/27/2022' '7/7/2022' '7/11/2022' '7/19/2022'
 '7/26/2022' '8/1/2022' '8/8/2022' '8/15/2022' '7/5/2022' '8/9/2022'
 '8/16/2022' '7/12/2022' '8/2/2022' '7/25/2022' '8/23/2022' '7/18/2022'
 '8/22/2022']
['index', 'Date', 'crop', 'soil type', 'target', 'spacing (m)', 'canopy_avg', 'MA_MeanAT', 'MA_MeanDP', 'MA_MeanRH', 'MA_MeanWS', 'MA_MeanIR', 'MA_MeanRH86', 'MA_MeanATD', 'MA_MeanRF_IBM', 'MA_MeanRF_LOC', 'MA_MeanPC_IBM', 'MA_MeanPC_LOC', 'MA_MeanSM_IR', 'MA_MeanST_IR', 'MA_MeanWet_count', 'MA_MeanDry_count', 'MA_MaxAT', 'MA_MaxDP', 'MA_MaxRH',

In [22]:
# 21 - 23 RTMA
RTMA_23_df = filtered_df.loc[:, ~filtered_df.columns.str.contains('LOC|PC|IR|count')]

# check years
unique_values = RTMA_23_df['Date'].unique()
print(unique_values)

# list headers
print(list(RTMA_23_df))

# save final dataset as csv
RTMA_23_df.to_csv(path + 'RTMA_23_df.csv', index=False)

['8/6/2021' '8/17/2021' '7/26/2021' '8/5/2021' '8/12/2021' '8/19/2021'
 '7/6/2021' '7/13/2021' '7/20/2021' '8/2/2021' '8/10/2021' '7/9/2021'
 '7/17/2021' '7/30/2021' '7/8/2021' '7/15/2021' '7/19/2021' '7/27/2021'
 '8/9/2021' '8/16/2021' '8/23/2021' '8/3/2021' '8/30/2021' '6/28/2022'
 '7/6/2022' '7/13/2022' '7/20/2022' '7/27/2022' '8/4/2022' '8/10/2022'
 '8/17/2022' '8/24/2022' '7/8/2022' '7/14/2022' '7/21/2022' '8/5/2022'
 '8/11/2022' '8/25/2022' '6/27/2022' '7/7/2022' '7/11/2022' '7/19/2022'
 '7/26/2022' '8/1/2022' '8/8/2022' '8/15/2022' '7/5/2022' '8/9/2022'
 '8/16/2022' '7/12/2022' '8/2/2022' '7/25/2022' '8/23/2022' '7/18/2022'
 '8/22/2022' '7/5/2023' '7/12/2023' '7/15/2023' '7/19/2023' '7/22/2023'
 '7/26/2023' '7/30/2023' '8/4/2023' '8/10/2023' '8/18/2023' '8/25/2023']
['index', 'Date', 'crop', 'soil type', 'target', 'spacing (m)', 'canopy_avg', 'MA_MeanAT', 'MA_MeanDP', 'MA_MeanRH', 'MA_MeanWS', 'MA_MeanRH86', 'MA_MeanATD', 'MA_MeanRF_IBM', 'MA_MaxAT', 'MA_MaxDP', 'MA_MaxRH', 'MA_

In [23]:
# 21 - 23 RTMA + LOCOMOS - irri/pcp
RTMA_LX_23_df = filtered_df.loc[:, ~filtered_df.columns.str.contains('LOC|PC|count')]
RTMA_LX_23_df = RTMA_LX_23_df.drop(['MA_MeanIR', 'MA_MaxIR', 'MA_SumIR'], axis = 1)

# check years
unique_values = RTMA_LX_23_df['Date'].unique()
print(unique_values)

# list headers
print(list(RTMA_LX_23_df))

# save final dataset as csv
RTMA_LX_23_df.to_csv(path + 'RTMA_LX_23_df.csv', index=False)

['8/6/2021' '8/17/2021' '7/26/2021' '8/5/2021' '8/12/2021' '8/19/2021'
 '7/6/2021' '7/13/2021' '7/20/2021' '8/2/2021' '8/10/2021' '7/9/2021'
 '7/17/2021' '7/30/2021' '7/8/2021' '7/15/2021' '7/19/2021' '7/27/2021'
 '8/9/2021' '8/16/2021' '8/23/2021' '8/3/2021' '8/30/2021' '6/28/2022'
 '7/6/2022' '7/13/2022' '7/20/2022' '7/27/2022' '8/4/2022' '8/10/2022'
 '8/17/2022' '8/24/2022' '7/8/2022' '7/14/2022' '7/21/2022' '8/5/2022'
 '8/11/2022' '8/25/2022' '6/27/2022' '7/7/2022' '7/11/2022' '7/19/2022'
 '7/26/2022' '8/1/2022' '8/8/2022' '8/15/2022' '7/5/2022' '8/9/2022'
 '8/16/2022' '7/12/2022' '8/2/2022' '7/25/2022' '8/23/2022' '7/18/2022'
 '8/22/2022' '7/5/2023' '7/12/2023' '7/15/2023' '7/19/2023' '7/22/2023'
 '7/26/2023' '7/30/2023' '8/4/2023' '8/10/2023' '8/18/2023' '8/25/2023']
['index', 'Date', 'crop', 'soil type', 'target', 'spacing (m)', 'canopy_avg', 'MA_MeanAT', 'MA_MeanDP', 'MA_MeanRH', 'MA_MeanWS', 'MA_MeanRH86', 'MA_MeanATD', 'MA_MeanRF_IBM', 'MA_MeanSM_IR', 'MA_MeanST_IR', 'MA_MaxA

In [24]:
# 21 - 23 RTMA + LOCOMOS
RTMA_LOC_23_df = filtered_df

# check years
unique_values = RTMA_LOC_23_df['Date'].unique()
print(unique_values)

# list headers
print(list(RTMA_LOC_23_df))

# save final dataset as csv
RTMA_LOC_23_df.to_csv(path + 'RTMA_LOC_23_df.csv', index=False)

['8/6/2021' '8/17/2021' '7/26/2021' '8/5/2021' '8/12/2021' '8/19/2021'
 '7/6/2021' '7/13/2021' '7/20/2021' '8/2/2021' '8/10/2021' '7/9/2021'
 '7/17/2021' '7/30/2021' '7/8/2021' '7/15/2021' '7/19/2021' '7/27/2021'
 '8/9/2021' '8/16/2021' '8/23/2021' '8/3/2021' '8/30/2021' '6/28/2022'
 '7/6/2022' '7/13/2022' '7/20/2022' '7/27/2022' '8/4/2022' '8/10/2022'
 '8/17/2022' '8/24/2022' '7/8/2022' '7/14/2022' '7/21/2022' '8/5/2022'
 '8/11/2022' '8/25/2022' '6/27/2022' '7/7/2022' '7/11/2022' '7/19/2022'
 '7/26/2022' '8/1/2022' '8/8/2022' '8/15/2022' '7/5/2022' '8/9/2022'
 '8/16/2022' '7/12/2022' '8/2/2022' '7/25/2022' '8/23/2022' '7/18/2022'
 '8/22/2022' '7/5/2023' '7/12/2023' '7/15/2023' '7/19/2023' '7/22/2023'
 '7/26/2023' '7/30/2023' '8/4/2023' '8/10/2023' '8/18/2023' '8/25/2023']
['index', 'Date', 'crop', 'soil type', 'target', 'spacing (m)', 'canopy_avg', 'MA_MeanAT', 'MA_MeanDP', 'MA_MeanRH', 'MA_MeanWS', 'MA_MeanIR', 'MA_MeanRH86', 'MA_MeanATD', 'MA_MeanRF_IBM', 'MA_MeanRF_LOC', 'MA_MeanPC